In [144]:
from sqlalchemy import Column, Integer, Numeric, String, Date, Index, Text, text, Float, create_engine  # The architects of our database schema, crafting the structure of our data haven.
from sqlalchemy.orm import sessionmaker, declarative_base  # Craftsmen who build the bridge between our Pythonic realm and the database underworld.
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.orm import declarative_base, sessionmaker

from rapidframes import RapiDF, QueryBuilder

import threading
import warnings
from typing import List, Dict
import polars as pl
import pandas as pd

In [ ]:
!pip3 install -r requirements.txt 

In [145]:
import sys
import os

sys.path.append('/home/ubuntu/OTB/rapidframes/')
sys.path.append('/home/ubuntu/OTB/BMAPS/routes')

In [146]:
import datetime
from collections import namedtuple
from datetime import datetime, timedelta, date
from time import ctime
from typing import Dict, List
import pandas as pd
import numpy as np
import polars as pl
from rapidframes import QueryBuilder, RapiDF
from warnings import simplefilter

In [147]:
STORAGE_URL = 'lmdb://databases/bmaps_data?map_size=50GB'
rf = RapiDF(STORAGE_URL)

[2024-04-05 07:02:22.824] [rapidframes] [warning] LMDB path at /home/ubuntu/OTB/databases/bmaps_data/ has already been opened in this process which is not supported by LMDB. You should only open a single RapiDF instance over a given LMDB path. To continue safely, you should delete this RapiDF instance and any others over the LMDB path in this process and then try again. Current process ID=[677138]


In [148]:
# Library RA_DATA got created
rf.get_library("RA_DATA").list_symbols()

[]

In [149]:
LIBRARY_NAME = "RA_DATA"

In [150]:
class PostgresDatabaseSecretsHolder:
    # Configuring the warehouse to store the contraband.
    USERNAME = "mohit"
    # PASSWORD = "password"
    PASSWORD = "password"
    HOST     = "localhost"
    PORT     = "5433"
    DATABASE = "bmaps"

# Establish a connection to the MariaDB and Postgre Server, like making an untraceable phone call to an old friend.
# Connecting to the MariaDB and Postgre Instances is akin to reaching out to a contact in a foreign land.
# The connection stringv, a secret formula for establishing a clandestine rendezvous.
# mariadb_connection_string = f"mysql+pymysql://{MariaDBDatabaseSecretsHolder.USERNAME}:{MariaDBDatabaseSecretsHolder.6eA1Wyp6l'sj}\
# @{MariaDBDatabaseSecretsHolder.HOST}:{MariaDBDatabaseSecretsHolder.PORT}/{MariaDBDatabaseSecretsHolder.DATABASE}"

postgres_connection_string = f"postgresql+psycopg2://{PostgresDatabaseSecretsHolder.USERNAME}:{PostgresDatabaseSecretsHolder.PASSWORD}\
@{PostgresDatabaseSecretsHolder.HOST}:{PostgresDatabaseSecretsHolder.PORT}/{PostgresDatabaseSecretsHolder.DATABASE}"

# Create the engines, the heart of our operation, silent yet powerful.
# mariadb_engine = create_engine(mariadb_connection_string)
postgres_engine = create_engine(postgres_connection_string)
# The blueprint of our data structure, meticulous and precise.
Base = declarative_base()
Session = sessionmaker(bind=postgres_engine)
session = Session()

class SalesDataTrnx(Base):
    __tablename__ = 'salesdata_trnx'
    ID               = Column(Integer, primary_key=True, autoincrement=True)
    INVENTLOCATIONID = Column(String(25))
    ITEMID           = Column(String(50))
    SALESQTY         = Column(Numeric(15, 3))
    LINEDISC         = Column(Numeric(15, 3))
    LINEAMOUNT       = Column(Numeric(15, 3))
    SALESPRICE       = Column(Numeric(15, 3))
    COSTPRICE        = Column(Numeric(15, 3))
    INVOICEDATE      = Column(Date())
    channel          = Column(String(50))
    family           = Column(String(30))

    __table_args__ = (
        Index('salesdata_trnx_INVOICEDATE_IDX', 'INVOICEDATE'),
        Index('salesdata_trnx_ITEMID_IDX', 'ITEMID', 'INVENTLOCATIONID', 'channel', 'family'),
        Index('salesdata_trnx_LINEAMOUNT_IDX', 'LINEAMOUNT'),
        Index('salesdata_trnx_INVENTLOCATIONID_1_IDX', 'INVENTLOCATIONID', 'ITEMID', 'channel'),
        Index('salesdata_trnx_INVENTLOCATIONID_2_IDX', 'INVENTLOCATIONID', 'channel', 'INVOICEDATE', 'ITEMID', 'family'),
        Index('salesdata_trnx_family_IDX', 'family', 'ITEMID', 'LINEAMOUNT'),
        Index('salesdata_trnx_family_amt_IDX', 'family', 'LINEAMOUNT'),
    )

class RATransactionTable(Base):
    __tablename__ = 'ra_transaction_table'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    Channel = Column(Text)
    country = Column(Text)
    Region = Column(Text)
    area = Column(Text)
    city = Column(Text)
    StoreId = Column(Integer)
    Store = Column(Text)
    season = Column(Text)
    Department = Column(Text)
    Family = Column(Text)
    SubFamily = Column(Text)
    Category = Column(Text)
    DOM_COMM = Column(Text)
    SubCategory = Column(Text)
    ExtendedSubCategory = Column(Text)
    Supplier = Column(Text)
    SubCategorySupplier = Column(Text)
    AssemblyCodeNickName = Column(Text)
    Status = Column(Text)
    ENDOFLife = Column(Text)
    Description = Column(Text)
    ItemLookupCode = Column(Text)
    BudgetYear = Column(Float)
    BDate = Column(Date)
    BudgetAmount = Column(Float)
    UnitsBuyBySku = Column(Numeric)
    BudgetGrossMarginP = Column(Float)
    BudgetCost = Column(Float)
    BudgetQTY = Column(Numeric)
    OpeningStock = Column(Numeric)
    ClosingStock = Column(Numeric)
    StockReceivedQty = Column(Integer)
    TotalPurchaseQty = Column(Numeric)
    StockOnHandQty = Column(Numeric)
    CurrentStockCost = Column(Float)
    budget_quarter = Column(Text)
    budget_month = Column(Text)
    budget_week = Column(Text)
    budget_day = Column(Text)

In [151]:
import logging
import psycopg2
from psycopg2 import OperationalError
import traceback

# Enable logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)

try:
    conn = psycopg2.connect(dbname="bmaps", user="mohit", host = "localhost", port = "5432", password = "password")
    conn.close()
    print(True)
except Exception as e:
    print(traceback.format_exc())
    print(e)

True


In [152]:
# When no fetch
lib = rf.get_library('RA_DATA', create_if_missing=True)

In [153]:
def fetch_item_data():
    lib = rf.get_library('RA_DATA', create_if_missing=True)
    print(f"[INFO] FETCH ITEM_MASTER TRANSACTIONS")
    
    ps_session = sessionmaker(bind=postgres_engine)
    ps_conn    = ps_session()
    item_data    = ps_conn.execute(text('''SELECT "ItemLookupCode" AS "ITEMID",
                                                 MAX("Description") AS "Description",
                                                 MAX("Department") AS "Department",
                                                 MAX("CategoryName") AS "Category",
                                                 MAX("Family") AS "Family",
                                                 MAX("subFamily") AS "SubFamily",
                                                 MAX("SubCategory") AS "SubCategory",
                                                 MAX("ExtendedSubCategory") AS "ExtendedSubCategory",
                                                 MAX("SubCategorySupplier") AS "SubCategorySupplier",
                                                 MAX("AssemblyCodeNickname") AS "AssemblyCodeNickName",
                                                 MAX("ENDOFLife") AS "ENDOFLife",
                                                 MAX("DOM_COMM") AS "DOM_COMM",
                                                 MAX("Status") AS "Status",
                                                 MAX("Supplier") AS "Supplier"
                                                 FROM 
                                                    itemmaster
                                                 GROUP BY 
                                                    "ItemLookupCode"'''))
    item_master      = pd.DataFrame(item_data, columns=item_data.keys())
    item_master["ENDOFLife"]  = item_master["ENDOFLife"].fillna("")
    item_master['ITEMID'] = item_master['ITEMID'].astype(str)

    lib.write("item_master", item_master, prune_previous_versions=True) # , prune_previous_versions=True
    return lib
lib = fetch_item_data()

[INFO] FETCH ITEM_MASTER TRANSACTIONS


In [154]:
q = QueryBuilder()
item_master = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('item_master', query_builder=q).data)
type(item_master.head(2))

polars.dataframe.frame.DataFrame

In [155]:
def get_type_name(val):
    return type(val).__name__

    # Apply the function to the column
# item_master = item_master.with_columns(pl.col('ITEMID').apply(get_type_name).alias('type_name'))

    # Count the values
    # item_master = item_master.groupby('type_name').agg(pl.count('type_name'))

In [156]:
# item_master['type_name'].value_counts()

In [157]:
def fetch_ra_data():
    lib = rf.get_library('RA_DATA', create_if_missing=True)

    # Fetch RA Transactions
    print(f"[INFO] FETCH RA TRANSACTIONS")
    
    ps_session = sessionmaker(bind=postgres_engine)
    ps_conn    = ps_session()
    ra_data    = ps_conn.execute(text("SELECT * from ra_table"))
    ra_df      = pd.DataFrame(ra_data, columns=ra_data.keys())
    ra_df['ITEMID'] = ra_df['ITEMID'].astype(str)
    ra_df["Budget_date"]      = pd.to_datetime(ra_df["Budget_date"])
    # ra_df["ENDOFLife"]  = ra_df["ENDOFLife"].fillna("")
    # cols_to_conv        = ["ClosingStock", "TotalPurchaseQty", "StockOnHandQty", "UnitsBuyBySku", "BudgetQTY", "OpeningStock"]
    # ra_df[cols_to_conv] = ra_df[cols_to_conv].apply(pd.to_numeric)

    lib.write("ra_trnx", ra_df, prune_previous_versions=True)
    ra_df = None

    return lib

lib = fetch_ra_data()

[INFO] FETCH RA TRANSACTIONS


In [158]:
q = QueryBuilder()
ra_trnx = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('ra_trnx', query_builder=q).data)

In [159]:
ra_trnx['Budget_date']

Budget_date
datetime[ns]
2024-03-28 00:00:00
2024-03-27 00:00:00
2024-03-28 00:00:00
2024-03-27 00:00:00
2024-03-28 00:00:00
2024-03-27 00:00:00
2024-03-28 00:00:00
2024-03-27 00:00:00
2024-03-28 00:00:00


In [160]:
ra_trnx.shape

(9792848, 11)

In [161]:
# ra_trnx = ra_trnx.with_columns(pl.col('ITEMID').apply(get_type_name).alias('type_name'))
# ra_trnx['type_name'].value_counts()

In [162]:
def fetch_cst_data():
    lib = rf.get_library('RA_DATA', create_if_missing=True)

    # Fetch CST Transactions
    print(f"[INFO] FETCH CURRENT SALES TRANSACTIONS")
    
    ps_session            = sessionmaker(bind=postgres_engine)
    ps_conn               = ps_session()
    cst_data              = ps_conn.execute(text("SELECT * from current_sales_trnx limit 100000;"))
    cst_df                = pd.DataFrame(cst_data, columns=cst_data.keys())
    cst_df["INVOICEDATE"] = pd.to_datetime(cst_df["INVOICEDATE"])
    cols_to_conv          = ["LINEAMOUNT", "SALESQTY", "COSTPRICE", "gross_sales", "historical_year"]
    cst_df[cols_to_conv]  = cst_df[cols_to_conv].apply(pd.to_numeric)
    
    lib.write("cst_trnx", cst_df, prune_previous_versions=True) # , prune_previous_versions=True

    return lib


lib = fetch_cst_data()

[INFO] FETCH CURRENT SALES TRANSACTIONS


In [209]:
q = QueryBuilder()
cst_trnx  = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('cst_trnx', query_builder=q).data)

In [210]:
cst_trnx.describe()


statistic,ITEMID,channel,INVENTLOCATIONID,LINEAMOUNT,SALESQTY,COSTPRICE,gross_sales,historical_year,INVOICEDATE,history_quarter,history_month,history_week,history_day,to_calculate_the_stock_till_now
str,str,str,str,f64,f64,f64,f64,f64,str,str,str,str,str,f64
"""count""","""100000""","""99986""","""100000""",100000.0,100000.0,100000.0,100000.0,100000.0,"""100000""","""100000""","""100000""","""100000""","""100000""",100000.0
"""null_count""","""0""","""14""","""0""",0.0,0.0,0.0,0.0,0.0,"""0""","""0""","""0""","""0""","""0""",0.0
"""mean""",null,null,null,3293.529308,1.65095,3076.613006,3320.047084,2021.95373,"""2022-06-14 09:…",null,null,null,null,0.0
"""std""",null,null,null,17172.741959,4.235381,8518.278807,17208.024918,0.745033,null,null,null,null,null,0.0
"""min""","""1ASHACWLYEX24U…","""""","""""",-1.166666e6,-200.0,0.0,-1.166666e6,2021.0,"""2021-01-01 00:…","""1""","""April ""","""1""","""Friday """,0.0
"""25%""",null,null,null,220.0,1.0,292.97,220.0,2021.0,"""2021-10-27 00:…",null,null,null,null,0.0
"""50%""",null,null,null,465.0,1.0,717.99,486.0,2022.0,"""2022-06-15 00:…",null,null,null,null,0.0
"""75%""",null,null,null,3039.05,2.0,3234.98,3100.0,2022.0,"""2022-12-10 00:…",null,null,null,null,0.0
"""max""","""1AULACWLUAP12I…","""WAREHOUSE""","""re-9014""",1.166666e6,200.0,696126.83,1.166666e6,2024.0,"""2024-02-07 00:…","""4""","""September""","""9""","""Wednesday""",0.0


In [165]:
lib

Library(s(config=LMDB(path=/home/ubuntu/OTB/databases/bmaps_data)), path=RA_DATA, storage=lmdb_storage)

In [211]:
cst_trnx['history_day'].value_counts()

history_day,count
str,u32
"""Saturday """,14694
"""Friday """,10127
"""Wednesday""",15471
"""Monday """,15219
"""Tuesday """,14878
"""Thursday """,15222
"""Sunday """,14389


In [166]:
def fetch_stock_data():
  lib = rf.get_library('RA_DATA', create_if_missing=True)

  # Get the opening stock
  print(f"[INFO] FETCH STOCK TRANSACTIONS")

  ps_session = sessionmaker(bind=postgres_engine)
  ps_conn    = ps_session()

  stock_data    = ps_conn.execute(text('''	SELECT "ItemLookupCode" AS "ITEMID" ,
                                          "StoreID" AS "INVENTLOCATIONID",
                                          SUM("Quantity") AS "stock_on_hand_qty",
                                          SUM("AvailableQTY") AS "opening_stock",
                                          SUM("AvailableQTY") AS "closing_stock",
                                          AVG("Price") AS "current_stock_cost_at_retail",
                                          0 AS "stock_received_qty",
                                          SUM("Cost") AS "opening_stock_at_cost",
                                          0 AS "adjustment_at_cost",
                                          0 AS "adjustment_at_retail",
                                          MAX("SnapShotTime") AS "stock_date"
                                          FROM stock 
                                          GROUP BY "ItemLookupCode","StoreID";
                                    '''
                                  ))
  stock_df      = pd.DataFrame(stock_data, columns=stock_data.keys())
  cols_to_conv        = ["stock_on_hand_qty", "opening_stock", "closing_stock", "current_stock_cost_at_retail"]
  stock_df[cols_to_conv] = stock_df[cols_to_conv].apply(pd.to_numeric)
  # problematic_rows = stock_df[pd.to_datetime(stock_df["stock_date"], errors='coerce').isna()]
  # print(problematic_rows, 'rows causing datetime err')
  # print('hiiii')
  stock_df["stock_date"] = pd.to_datetime(stock_df["stock_date"], errors='coerce')

  lib.write("stock_trnx", stock_df, prune_previous_versions=True)
  # stock_df = None
  return lib
lib = fetch_stock_data()

[INFO] FETCH STOCK TRANSACTIONS


In [167]:
# type(stock_df['stock_date'].dtypes)

In [168]:
q = QueryBuilder()
stock_trnx = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('stock_trnx', query_builder=q).data)

In [169]:
stock_trnx['stock_date'].value_counts()

stock_date,count
datetime[ns],u32
2024-02-26 00:00:00,112012


In [170]:
maximum_stock_available_date = datetime.date(stock_trnx['stock_date'].dt.max())
today_is = date.today()
print(f"INFO OF CLOSING IS AVAILABLE TILL :{maximum_stock_available_date} AND CURRENT DATE IS: {today_is}")

INFO OF CLOSING IS AVAILABLE TILL :2024-02-26 AND CURRENT DATE IS: 2024-04-05


In [171]:
if maximum_stock_available_date < date.today(): 
    cst_trnx = cst_trnx.with_columns((pl.when(cst_trnx['INVOICEDATE'].cast(pl.Date) > maximum_stock_available_date, cst_trnx['INVOICEDATE'].cast(pl.Date) < today_is)
                                                    .then(cst_trnx['SALESQTY'])
                                                    .otherwise(0)).alias('to_calculate_the_stock_till_now'))

In [172]:
sales_df['INVOICEDATE'].isnull().sum()          

0

In [173]:
sales_df = cst_trnx.to_pandas()

In [174]:
# sales_df.head()
lib.write("cst_trnx", sales_df, prune_previous_versions=True)

VersionedItem(symbol='cst_trnx', library='RA_DATA', data=n/a, version=1, metadata=None, host='LMDB(path=/home/ubuntu/OTB/databases/bmaps_data)')

In [175]:
stock_trnx.shape

(112012, 11)

In [176]:
type(stock_trnx['stock_date'][0])
stock_trnx['stock_date'][0]
datetime.date(stock_trnx['stock_date'][0])
stock_trnx['stock_date'].dt.max()

datetime.datetime(2024, 2, 26, 0, 0)

In [177]:
stock_trnx.shape

(112012, 11)

In [178]:
def fetch_kpi_data():
    lib = rf.get_library('RA_DATA', create_if_missing=True)

    # Get the opening stock
    print(f"[INFO] FETCH KPI TRANSACTIONS")

    ps_session = sessionmaker(bind=postgres_engine)
    ps_conn    = ps_session()

    kpi_data    = ps_conn.execute(text("SELECT * from kpi_table;"))
    kpi_df      = pd.DataFrame(kpi_data, columns=kpi_data.keys())

    # ra_df["Budget_date"]      = pd.to_datetime(ra_df["Budget_date"])
    # ra_df["ENDOFLife"]  = ra_df["ENDOFLife"].fillna("")
    # cols_to_conv        = ["budget_qty"]

    # cols_to_conv        = ["ClosingStock", "TotalPurchaseQty", "StockOnHandQty", "UnitsBuyBySku", "BudgetQTY", "OpeningStock"]
    # ra_df[cols_to_conv] = ra_df[cols_to_conv].apply(pd.to_numeric)
  
    lib.write("kpi_trnx", kpi_df, prune_previous_versions=True)#,prune_previous_versions=True
    return lib
lib = fetch_kpi_data()


[INFO] FETCH KPI TRANSACTIONS


In [179]:
q = QueryBuilder()
kpi_trnx  = pl.DataFrame(rf.get_library(LIBRARY_NAME).read("kpi_trnx", query_builder=q).data)

In [180]:
kpi_trnx.shape

(2847, 15)

In [181]:
rf.list_libraries()

['RA_DATA', 'SP_DATA']

In [183]:
print(*map(lambda item:(print(item), item)[1], lib.list_versions().items()), 'the symbol version')

(stock_trnx_v0, (date=2024-04-05 07:04:04.818796748+00:00))
(ra_trnx_v0, (date=2024-04-05 07:03:18.343212761+00:00))
(kpi_trnx_v0, (date=2024-04-05 07:04:12.068450644+00:00))
(item_master_v0, (date=2024-04-05 07:02:28.163985199+00:00))
(cst_trnx_v1, (date=2024-04-05 07:04:09.275053066+00:00))
(stock_trnx_v0, (date=2024-04-05 07:04:04.818796748+00:00)) (ra_trnx_v0, (date=2024-04-05 07:03:18.343212761+00:00)) (kpi_trnx_v0, (date=2024-04-05 07:04:12.068450644+00:00)) (item_master_v0, (date=2024-04-05 07:02:28.163985199+00:00)) (cst_trnx_v1, (date=2024-04-05 07:04:09.275053066+00:00)) the symbol version


In [184]:
import rapidframes
class rapidframe_symbols:
    #Class attributes
    ITEM_MASTER  = 'item_master'
    RA_TRNX      = 'ra_trnx'
    CST_TRNX     = 'cst_trnx'
    STOCK_TRNX   = 'stock_trnx'
    KPI_TRNX     = 'kpi_trnx'
    # Join the symbols
    ITEM_RA      = 'item_ra_joined'
    RA_STOCK     = 'ra_stock_joined'
    STOCK_CST    = 'stock_cst_joined'
    CST_KPI      = 'cst_kpi_joined'
    
    def combine_the_rapid_frame_symbols(self, rf : rapidframes.repidf.RapiDF, Stack : str, First, Last, LIBRARY_NAME, join_element):
        
        rf.get_library(LIBRARY_NAME).write(
            Stack,
            First.join(Last,
            on=join_element,
            how='left' 
            ).to_pandas(), prune_previous_versions=True
        )
# symbols = rapidframe_symbols()

In [185]:
# symbols.combine_the_rapid_frame_symbols(rf, symbols.ITEM_RA, item_master, ra_trnx, LIBRARY_NAME, list('ITEMID'))

In [186]:
ra_trnx

Store,area,Region,adjusted_budget_gross_margin_percent,budget_amount,budget_cost,budget_qty,ITEMID,Channel,INVENTLOCATIONID,Budget_date
str,str,str,f64,f64,f64,f64,str,str,str,datetime[ns]
"""Shoubra Branch…","""unknown""","""unknown""",20.0,307.116348,245.693079,3.0,"""1SBBPWHC00ST39…","""Retail""","""Re-1023""",2024-03-28 00:00:00
"""Gesr El Suez B…","""unknown""","""unknown""",20.0,0.0,-0.0,3.0,"""1SBBPWHC00ST39…","""Retail""","""Re-1045""",2024-03-27 00:00:00
"""Gesr El Suez B…","""unknown""","""unknown""",20.0,0.0,-0.0,3.0,"""1SBBPWHC00ST39…","""Retail""","""Re-1045""",2024-03-28 00:00:00
"""Asafra Branch""","""unknown""","""unknown""",20.0,40.804668,32.643735,3.0,"""1SBBPWHC00ST39…","""Retail""","""Re-2011""",2024-03-27 00:00:00
"""Asafra Branch""","""unknown""","""unknown""",20.0,40.804668,32.643735,3.0,"""1SBBPWHC00ST39…","""Retail""","""Re-2011""",2024-03-28 00:00:00
"""Mega Alex Sidi…","""Alex""","""ALEXANDRIA""",20.0,588.349469,470.679575,3.0,"""1SBBPWHC00ST39…","""Retail""","""Re-2022""",2024-03-27 00:00:00
"""Mega Alex Sidi…","""Alex""","""ALEXANDRIA""",20.0,588.349469,470.679575,3.0,"""1SBBPWHC00ST39…","""Retail""","""Re-2022""",2024-03-28 00:00:00
"""El Sadat Branc…","""unknown""","""unknown""",20.0,116.839758,93.471807,3.0,"""1SBBPWHC00ST39…","""Retail""","""Re-3020""",2024-03-27 00:00:00
"""El Sadat Branc…","""unknown""","""unknown""",20.0,116.839758,93.471807,3.0,"""1SBBPWHC00ST39…","""Retail""","""Re-3020""",2024-03-28 00:00:00


In [187]:
rf.get_library(LIBRARY_NAME).append(
            'ra_item_joined',
            ra_trnx.join(item_master,
            on=['ITEMID'],
            how='left' 
            ).to_pandas(), prune_previous_versions=True
        )

VersionedItem(symbol='ra_item_joined', library='RA_DATA', data=n/a, version=0, metadata=None, host='LMDB(path=/home/ubuntu/OTB/databases/bmaps_data)')

In [188]:
q=QueryBuilder()
ra_item_joined = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('ra_item_joined', query_builder=q).data)

In [189]:
ra_item_joined.shape

(9792848, 24)

In [190]:
rf.get_library(LIBRARY_NAME).append(
            'ra_stock_joined',
            ra_item_joined.join(stock_trnx,
            on=['ITEMID', 'INVENTLOCATIONID'],
            how='left' 
            ).to_pandas(), prune_previous_versions=True
        )

VersionedItem(symbol='ra_stock_joined', library='RA_DATA', data=n/a, version=0, metadata=None, host='LMDB(path=/home/ubuntu/OTB/databases/bmaps_data)')

In [191]:
q=QueryBuilder()
ra_stock_joined = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('ra_stock_joined', query_builder=q).data)
# ra_stock_joined.dtypes

In [192]:
stock_trnx['stock_date'].value_counts()

stock_date,count
datetime[ns],u32
2024-02-26 00:00:00,112012


In [193]:
ra_stock_joined = ra_stock_joined.with_columns(pl.col('stock_date').fill_null(datetime(2024,2,26,00,00,00)))

In [194]:
cst_trnx['INVOICEDATE'].cast(pl.Date)[0]
cst_trnx['INVOICEDATE'].dt.min()

datetime.datetime(2021, 1, 1, 0, 0)

In [195]:
cst_trnx['historical_year']

historical_year
i64
2021
2021
2021
2021
2021
2021
2021
2021
2021


In [196]:
cst_trnx.melt(variable_name="column").group_by(pl.all()).len()['column'].to_list()

['gross_sales',
 'LINEAMOUNT',
 'LINEAMOUNT',
 'LINEAMOUNT',
 'gross_sales',
 'gross_sales',
 'gross_sales',
 'INVOICEDATE',
 'history_month',
 'LINEAMOUNT',
 'COSTPRICE',
 'COSTPRICE',
 'LINEAMOUNT',
 'LINEAMOUNT',
 'LINEAMOUNT',
 'gross_sales',
 'LINEAMOUNT',
 'LINEAMOUNT',
 'gross_sales',
 'gross_sales',
 'LINEAMOUNT',
 'COSTPRICE',
 'LINEAMOUNT',
 'COSTPRICE',
 'LINEAMOUNT',
 'gross_sales',
 'gross_sales',
 'LINEAMOUNT',
 'gross_sales',
 'LINEAMOUNT',
 'LINEAMOUNT',
 'COSTPRICE',
 'INVOICEDATE',
 'LINEAMOUNT',
 'gross_sales',
 'gross_sales',
 'LINEAMOUNT',
 'LINEAMOUNT',
 'LINEAMOUNT',
 'COSTPRICE',
 'LINEAMOUNT',
 'gross_sales',
 'gross_sales',
 'gross_sales',
 'gross_sales',
 'COSTPRICE',
 'gross_sales',
 'COSTPRICE',
 'INVOICEDATE',
 'gross_sales',
 'LINEAMOUNT',
 'LINEAMOUNT',
 'COSTPRICE',
 'gross_sales',
 'LINEAMOUNT',
 'INVOICEDATE',
 'LINEAMOUNT',
 'gross_sales',
 'LINEAMOUNT',
 'LINEAMOUNT',
 'gross_sales',
 'LINEAMOUNT',
 'COSTPRICE',
 'gross_sales',
 'LINEAMOUNT',
 'LINE

In [197]:
rf.get_library(LIBRARY_NAME).append(
            'ra_kpi_joined',
            ra_stock_joined.join(kpi_trnx,
            on=['ITEMID'],
            how='left' 
            ).to_pandas(), prune_previous_versions=True
        )

VersionedItem(symbol='ra_kpi_joined', library='RA_DATA', data=n/a, version=0, metadata=None, host='LMDB(path=/home/ubuntu/OTB/databases/bmaps_data)')

In [198]:
q=QueryBuilder()
ra_kpi_joined  = pl.DataFrame(rf.get_library(LIBRARY_NAME).read('ra_kpi_joined', query_builder=q).data)

In [199]:
ra_kpi_joined.shape

(9792848, 47)

In [200]:
ra_kpi_joined['stock_date'].value_counts()

stock_date,count
datetime[ns],u32
2024-02-26 00:00:00,9792848


In [201]:
fill_date = ra_kpi_joined['stock_date'].dt.max()
ra_kpi_joined = ra_kpi_joined.with_columns(stock_date = pl.col('stock_date').fill_null(fill_date))
lib.write('ra_kpi_joined', ra_kpi_joined.to_pandas(), prune_previous_versions=True)

VersionedItem(symbol='ra_kpi_joined', library='RA_DATA', data=n/a, version=1, metadata=None, host='LMDB(path=/home/ubuntu/OTB/databases/bmaps_data)')

In [202]:
print(*map(lambda item:(print(item), item)[1], dict(zip(ra_kpi_joined.columns, ra_kpi_joined.dtypes)).items()), 'the symbol version')

('Store', String)
('area', String)
('Region', String)
('adjusted_budget_gross_margin_percent', Float64)
('budget_amount', Float64)
('budget_cost', Float64)
('budget_qty', Float64)
('ITEMID', String)
('Channel', String)
('INVENTLOCATIONID', String)
('Budget_date', Datetime(time_unit='ns', time_zone=None))
('Description', String)
('Department', String)
('Category', String)
('Family', String)
('SubFamily', String)
('SubCategory', String)
('ExtendedSubCategory', String)
('SubCategorySupplier', String)
('AssemblyCodeNickName', String)
('ENDOFLife', String)
('DOM_COMM', String)
('Status', String)
('Supplier', String)
('stock_on_hand_qty', Float64)
('opening_stock', Float64)
('closing_stock', Float64)
('current_stock_cost_at_retail', Float64)
('stock_received_qty', Float64)
('opening_stock_at_cost', Float64)
('adjustment_at_cost', Float64)
('adjustment_at_retail', Float64)
('stock_date', Datetime(time_unit='us', time_zone=None))
('article_score_sale', Float64)
('article_score_abc', Float64)
(

In [134]:
def filter_symbol(rf: RapiDF, library_name: str, symbol_name: str, columns: List[str]=None, filter_set: Dict[str, str | int | list]= {}, return_as_pl: bool=True):
    # Initialize the QueryBuilder
    q = QueryBuilder()

    # Apply filters dynamically
    for column, conditions in filter_set.items():
        for operator, value in conditions.items():
            if operator == "_gt_": # Greater Than
                q = q[(q[column] > value)]
            elif operator == "_gte_": # Greater than or Equal To
                q = q[(q[column] >= pd.to_datetime(value))]
            elif operator == "_lt_": # Less Than
                q = q[(q[column] < value)]
            elif operator == "_lte_": # Less Than or Equal To
                q = q[(q[column] <= pd.to_datetime(value))]
            elif operator == "_eq_": # Equal To
                q = q[(q[column] == value)]
            elif operator == "_isin_": # Is in the list
                q = q[(q[column].isin(value))]

    filtered_df = rf.get_library(library_name).read(symbol_name, columns=columns, query_builder=q).data
    return pl.DataFrame(filtered_df) if return_as_pl else filtered_df


In [135]:
history_start_date_str  = "2021-01-01"
history_end_date_str    = "2023-12-16"

forecast_start_date_str = "2024-01-01"
forecast_end_date_str   = "2024-12-31"


In [136]:
list(ra_kpi_joined.columns)

['Store',
 'area',
 'Region',
 'adjusted_budget_gross_margin_percent',
 'budget_amount',
 'budget_cost',
 'budget_qty',
 'ITEMID',
 'Channel',
 'INVENTLOCATIONID',
 'Budget_date',
 'Description',
 'Department',
 'Category',
 'Family',
 'SubFamily',
 'SubCategory',
 'ExtendedSubCategory',
 'SubCategorySupplier',
 'AssemblyCodeNickName',
 'ENDOFLife',
 'DOM_COMM',
 'Status',
 'Supplier',
 'stock_on_hand_qty',
 'opening_stock',
 'closing_stock',
 'current_stock_cost_at_retail',
 'stock_received_qty',
 'opening_stock_at_cost',
 'adjustment_at_cost',
 'adjustment_at_retail',
 'stock_date',
 'article_score_sale',
 'article_score_abc',
 'article_score_ae',
 'article_score_speed',
 'article_score_terminal',
 'article_score_margin',
 'article_score_sell',
 'article_score_markdown',
 'article_score_core',
 'article_score_quartile',
 'article_score_sortimeter',
 'btech_vs_sortimeter',
 'store_sku_count',
 'price']

In [203]:
ra_kpi_joined['Budget_date'].value_counts()

Budget_date,count
datetime[ns],u32
2024-03-28 00:00:00,4896424
2024-03-27 00:00:00,4896424


In [204]:
ra_kpi_joined['Budget_date'][0]

datetime.datetime(2024, 3, 28, 0, 0)

In [205]:
ra_kpi_joined = ra_kpi_joined.with_columns((pl.when(ra_kpi_joined['Budget_date']==datetime(2024,3,27,0,0))
                                            .then(datetime(2024,4,5,00,00,00))
                                            .otherwise(datetime(2024,4,6,00,00,00))).alias('Budget_date'))

In [206]:
ra_kpi_joined = ra_kpi_joined.fill_null(0)

In [207]:
lib.append('ra_kpi_joined', ra_kpi_joined.to_pandas(), prune_previous_versions=True)

VersionedItem(symbol='ra_kpi_joined', library='RA_DATA', data=n/a, version=2, metadata=None, host='LMDB(path=/home/ubuntu/OTB/databases/bmaps_data)')

In [19]:
%reset -f